<a href="https://colab.research.google.com/github/seyoumyg/Southern-CA-Lake-Weather/blob/main/LakeWeather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import numpy as np
import pandas as pd
from ipywidgets import widgets
import plotly.graph_objects as go
from ipywidgets import Dropdown, VBox, HBox
from IPython.display import display


# New Section

In [2]:
# Read data set
df1 = pd.read_csv ("Daily_179.csv")
df2 = pd.read_csv ("Daily_192.csv")
df3 = pd.read_csv ("Daily_204.csv")
df4 = pd.read_csv ("Daily_240.csv")
df = pd.concat([df1,df2,df3,df4])


FileNotFoundError: [Errno 2] No such file or directory: 'Daily_179.csv'

In [ ]:
target_cols = ['Stn Name', 'Date', 'Precip (in)', 'Max Air Temp (F)', 'Min Air Temp (F)', 'Avg Air Temp (F)', 'Avg Rel Hum (%)','Avg Wind Speed (mph)']
dff= df[target_cols]
#dff['Date'] = pd.to_datetime(dff['Date'], format='%m/%d/%Y').dt.floor('D')
df_long = pd.melt(dff, id_vars=['Stn Name', 'Date'],
                  value_vars=['Precip (in)', 'Max Air Temp (F)', 'Min Air Temp (F)', 'Avg Air Temp (F)', 'Avg Rel Hum (%)','Avg Wind Speed (mph)']).dropna()

In [ ]:
station_dropdown = widgets.Dropdown(
    options=df_long['Stn Name'].unique().tolist(),
    value='Diamond Valley Lake',
    description='Lake:'
)

variable_dropdown = widgets.Dropdown(
    options=df_long['variable'].unique().tolist(),
    value='Precip (in)',
    description='Variable:'
)



In [ ]:
# Update plot function
def update_plot(*args):
    source = station_dropdown.value
    parameter = variable_dropdown.value

    filtered = df_long[(df_long['Stn Name'] == source) & (df_long['variable'] == parameter)][['Date', 'value']]
        #.sort_values('Date')
    #filtered['Date'] = pd.to_datetime(filtered['Date'])
    with fig.batch_update():
        fig.data[0].x = list(filtered['Date'])
        fig.data[0].y = list(filtered['value'])
        fig.layout.yaxis.title = parameter
        fig.layout.title = f'{parameter} from {source}'

In [ ]:
# Observe dropdown changes
station_dropdown.observe(update_plot, names='value')
variable_dropdown.observe(update_plot, names='value')
#zoom_slider.observe(update_zoom, names='value')

In [ ]:
# Create FigureWidget
fig = go.FigureWidget()
fig.add_scatter(mode='lines')
fig.update_layout(width=600, height=400,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7,
                     label="1 week", # The 'step' is day, but the 'count' makes it a week
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1 m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6 m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1 year",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Initial render

update_plot()
#fig.update_xaxes(
#    tickformat="%Y-%m-%d"  # Customize the date format here
#)
# Display with dropdowns on top
ui = VBox([
    HBox([station_dropdown, variable_dropdown]),
    fig
])
display(ui)